In [7]:
text = "これはサンプルテキストです"
prompt_text = "元気に"
prompt_tokens_path = "/nfs1/s1f102201582/projects/mhcc-moshi/moshi/fake.npy"
semantic_path = "./temp/codes_0.npy"
out_path = "./temp/fake.wav"
fs_tmp_path = "./temp"

In [8]:
import asyncio
import time
import functools

def task_B(task: asyncio.Task):
    """動的に追加されるタスク"""
    print(task.result())

async def task_A(semaphore, i):
    async with semaphore:
        print(f"start process: {i}")
        cmd = f"""python /nfs1/s1f102201582/projects/fish-speech/fish_speech/models/text2semantic/inference.py --text {text} --prompt-text {prompt_text} --prompt-tokens {prompt_tokens_path} --output-dir {fs_tmp_path} --checkpoint-path /nfs1/s1f102201582/projects/fish-speech/checkpoints/openaudio-s1-mini/"""
        print(cmd)
        process = await asyncio.create_subprocess_shell(
            cmd,
            stdout=asyncio.subprocess.PIPE,
            stderr=asyncio.subprocess.PIPE
        )
        stdout, stderr = await process.communicate()
    return stdout, stderr

async def main():
    semaphore = asyncio.Semaphore(5)
    all_tasks = []
    for i in range(10):
        cor = task_A(semaphore, i)
        task = asyncio.create_task(cor)
        task.add_done_callback(functools.partial(task_B))
        all_tasks.append(task)
    await asyncio.gather(*all_tasks)

In [9]:
import os
os.environ["PYTHONAYNCIODEBUG"] = "1"

print(f"[{time.strftime('%X')}] asyncio.run(main()) を呼び出します")
start_run = time.time()
await main()
end_run = time.time()
print(f"[{time.strftime('%X')}] asyncio.run() が終了しました (実行時間: {end_run - start_run:.2f}s)")

[22:12:56] asyncio.run(main()) を呼び出します
start process: 0
python /nfs1/s1f102201582/projects/fish-speech/fish_speech/models/text2semantic/inference.py --text これはサンプルテキストです --prompt-text 元気に --prompt-tokens /nfs1/s1f102201582/projects/mhcc-moshi/moshi/fake.npy --output-dir ./temp --checkpoint-path /nfs1/s1f102201582/projects/fish-speech/checkpoints/openaudio-s1-mini/
start process: 1
python /nfs1/s1f102201582/projects/fish-speech/fish_speech/models/text2semantic/inference.py --text これはサンプルテキストです --prompt-text 元気に --prompt-tokens /nfs1/s1f102201582/projects/mhcc-moshi/moshi/fake.npy --output-dir ./temp --checkpoint-path /nfs1/s1f102201582/projects/fish-speech/checkpoints/openaudio-s1-mini/
start process: 2
python /nfs1/s1f102201582/projects/fish-speech/fish_speech/models/text2semantic/inference.py --text これはサンプルテキストです --prompt-text 元気に --prompt-tokens /nfs1/s1f102201582/projects/mhcc-moshi/moshi/fake.npy --output-dir ./temp --checkpoint-path /nfs1/s1f102201582/projects/fish-speech/checkpoi